# Import Train Data from large movie set review (stanford)
[source](https://ai.stanford.edu/~amaas/data/sentiment/)

In [1]:
import glob
import pandas as pd

In [2]:
train_pos = []
path = "/home/mohit/sdb1_mount/New_CV/Learning/nlp_learning/course/text classification/aclImdb/train/pos/"
for files in glob.glob(path +"*.txt"):
    infile = open(files)
    a = infile.readline()
#     print(a)
    infile.close()
    train_pos.append(a)
    
train_neg = []
path = "/home/mohit/sdb1_mount/New_CV/Learning/nlp_learning/course/text classification/aclImdb/train/neg/"
for files in glob.glob(path +"*.txt"):
    infile = open(files)
    a = infile.readline()
#     print(a)
    infile.close()
    train_neg.append(a)

In [3]:
len(train_pos)

12500

In [4]:
train_pos[0]

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [5]:
train_neg[0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [6]:
len(train_pos)

12500

In [7]:
len(train_neg)

12500

In [8]:
#Data Preprocessing
import string
import re
def preprocess(text):
    text= text.lower()
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    t=text.translate(translator)
    return(re.sub(' +', ' ',t))

# Basic Approach Towards Text Classification: Let's Count

In this approach we are going to count the number of positive and negative words and accordingly we can decide the polarity of a review.
For example,
- If a review contains <b>5 positive words</b> and <b>2 negative words</b>, we can consider that review as <b><u>Positive</u></b>
- If a review contains <b>5 negative words</b> and <b>2 positive words</b>, we can consider that review as <b><u>Negative</u></b>

We have taken the list of positive an dnegative words from www.cs.uic.edu feel free to get your own list from this [source](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html)

In [9]:
#Loading the list of Positive Words:
path_pos = "/home/mohit/sdb1_mount/New_CV/Learning/nlp_learning/course/text classification/opinion-lexicon-English/positive-words.txt"
with open(path_pos) as f:
    pos_words = f.read().splitlines() 

#Loading the list of Negative Words:
path_neg = "/home/mohit/sdb1_mount/New_CV/Learning/nlp_learning/course/text classification/opinion-lexicon-English/negative-words.txt"
with open(path_neg) as f:
    neg_words = f.read().splitlines()  

In [10]:
pos_words[:5]

['a+', 'abound', 'abounds', 'abundance', 'abundant']

In [11]:
neg_words[:5]

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable']

In [12]:
#Function to find the polarity of given sentence:
def getPolarity(txt):
    txt_pre = preprocess(txt)
    positive_counter = 0
    negative_counter = 0
    s = txt_pre.split()
    for j in range(len(s)):
        if s[j] in pos_words:
            positive_counter += 1
        elif s[j] in neg_words:
            negative_counter += 1
    if positive_counter > negative_counter:
        return 1
    elif positive_counter < negative_counter:
        return -1
    else:
        return 0

print(getPolarity(train_pos[4]))
print(getPolarity(train_neg[4]))

1
-1


In [13]:
#Let's check the performance of this approach on the entire dataset:
total_score_pos = []
for i in range(len(train_pos)):
    scr = getPolarity(train_pos[i])
    total_score_pos.append(scr)

In [14]:
sum(total_score_pos)/len(total_score_pos)

0.55072

In [15]:
total_score_neg = []
for i in range(len(train_neg)):
    scr = getPolarity(train_neg[i])
    total_score_neg.append(scr)

In [16]:
sum(total_score_neg)/len(total_score_neg)

-0.36416

It worked descently, but the questions we need to answer are:
- is it scalable?
- can it be applied to other classification tasks?
- what should we do if we want to increase the accuracy?

In [17]:
len(train_pos)

12500

# Targetting Text Classification Through Modelling Approach

In order to approach this problem more precisely we will go ahead with the modelling approach. As none of the ML model accept text as input, we are going to prepare data in numerical format. There are a few basic steps that we would follow:

- Remove Stopwords
- Select 'N' Frequent Words
- Prepare Data in Numerical Format

## Bag of Words Approach
Here we will represent our text in a form of N words and corresponding count. This is called bag of words as the order doesn't matter.
For example, consider the following sentence <b>I love reading books but I also love playing</b> 

If we want to represent the above sentence as bag of words, we could do something as follows:

{I:2, love:2, reading:1, books:1, but:1, also:1, playing:1}

In [18]:
import nltk
from nltk.corpus import stopwords

stp = stopwords.words('english')

word_count ={}

#Process positive samples
for i in range(len(train_pos)):
    t_p = preprocess(train_pos[i].lower())
    s=t_p.split()
    for j in range(len(s)):
        if s[j] not in stp:
            if s[j] in word_count:
                word_count[s[j]] += 1
            else:
                word_count[s[j]] = 1
                
#Process negative samples
for i in range(len(train_neg)):
    t_p = preprocess(train_neg[i].lower())
    s=t_p.split()
    for j in range(len(s)):
        if s[j] not in stp:
            if s[j] in word_count:
                word_count[s[j]] += 1
            else:
                word_count[s[j]] = 1

In [19]:
word_count_sorted =dict(sorted(word_count.items(), key=lambda x: x[1], reverse=True))

In [20]:
#Keep top n words
N=500
l = range(N)
word_count_topN ={}
for k in l:
    word_count_topN[list(word_count_sorted.keys())[k]] = list(word_count_sorted.values())[k]

In [21]:
#Let's create dataframe out of it:
train_data_frame = pd.DataFrame(columns=word_count_topN.keys())

In [22]:
for i in range(len(train_pos)):
    train_data_frame.loc[i,:] = 0
    s = preprocess(train_pos[i])
    s = s.split()
    for j in range(len(s)):
        if s[j] in word_count_topN:
            train_data_frame.loc[i,s[j]] += 1
    if(i%1000==0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [23]:
#Let's tag them as Positive Sentiment
train_data_frame['sentiment'] = 1

In [24]:
len(train_neg)

12500

In [25]:
c = len(train_pos)
for i in range(len(train_neg)):
    train_data_frame.loc[c+i,:] = 0
    s = preprocess(train_neg[i])
    s = s.split()
    for j in range(len(s)):
        if s[j] in word_count_topN:
            train_data_frame.loc[c+i,s[j]] += 1
    if(i%1000==0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [26]:
train_data_frame.tail()

,br,movie,film,one,like,good,time,even,would,story,...,serious,reality,taken,whose,released,english,cinematography,ends,usually,sentiment
24995,0.0,0.0,3.0,4.0,0.0,1.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24996,12.0,1.0,7.0,4.0,0.0,1.0,2.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,3.0,4.0,0.0,1.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24998,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24999,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Let's train the model

We are going to use Keras for building this basic model

In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils, to_categorical
import pandas as pd
from sklearn.utils import shuffle
from keras import regularizers

train_data_frame = shuffle(train_data_frame)
X = train_data_frame.iloc[:,:-1]
y = train_data_frame.iloc[:, -1]
y = to_categorical(y)

Using TensorFlow backend.


In [28]:
X.shape

(25000, 500)

In [29]:
y.shape

(25000, 2)

In [30]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)



model = Sequential()
model.add(Dense(128, input_shape=(500,)))
model.add(Activation('sigmoid'))

model.add(Dense(64,kernel_regularizer=regularizers.l2(0.02)))
model.add(Activation('sigmoid'))

model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.fit(train_X, train_y, epochs=50, batch_size=512, verbose=1)

loss, accuracy = model.evaluate(test_X, test_y, verbose=0)

Epoch 1/50
20000/20000 [==============================] - 0s 23us/step - loss: 1.9561 - acc: 0.5763
Epoch 2/50
20000/20000 [==============================] - 0s 10us/step - loss: 1.2216 - acc: 0.7612
Epoch 3/50
20000/20000 [==============================] - 0s 9us/step - loss: 0.8008 - acc: 0.8092
Epoch 4/50
20000/20000 [==============================] - 0s 10us/step - loss: 0.5898 - acc: 0.8251
Epoch 5/50
20000/20000 [==============================] - 0s 10us/step - loss: 0.4924 - acc: 0.8321
Epoch 6/50
20000/20000 [==============================] - 0s 10us/step - loss: 0.4446 - acc: 0.8395
Epoch 7/50
20000/20000 [==============================] - 0s 10us/step - loss: 0.4221 - acc: 0.8410
Epoch 8/50
20000/20000 [==============================] - 0s 10us/step - loss: 0.4079 - acc: 0.8437
Epoch 9/50
20000/20000 [==============================] - 0s 10us/step - loss: 0.4009 - acc: 0.8440
Epoch 10/50
20000/20000 [==============================] - 0s 9us/step - loss: 0.3938 - acc: 0.8457
E

In [31]:
accuracy

0.8406

# Model Building using Tf-Idf Vectorizer

tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. tf–idf is one of the most popular term-weighting schemes today; 83% of text-based recommender systems in digital libraries use tf–idf.

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

<b>TF: Term Frequency,</b> which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: 

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

<img src="files/tf.png"/>

<b>IDF: Inverse Document Frequency,</b> which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: 

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

<img src="files/idf.png"/>

For Example,
<img src="files/example.png"/>


In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [33]:
train_full = train_pos+train_neg

In [34]:
preprocessor = lambda x: preprocess(x)
train_full_p = list(map(preprocessor, train_full))

In [35]:
train_full_p[0]

'bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high s satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i m here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn t '

In [53]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words=stp, min_df=0.02) # You can still specify n-grams here.

In [54]:
tfidf = vectorizer.fit_transform(train_full_p)

In [55]:
nms=vectorizer.get_feature_names()

In [56]:
nms

['10',
 '20',
 '30',
 '80',
 'able',
 'absolutely',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'actor',
 'actors',
 'actress',
 'actual',
 'actually',
 'add',
 'admit',
 'age',
 'ago',
 'agree',
 'air',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'amazing',
 'america',
 'american',
 'among',
 'animation',
 'annoying',
 'another',
 'anyone',
 'anything',
 'anyway',
 'apart',
 'apparently',
 'appear',
 'appears',
 'around',
 'art',
 'ask',
 'atmosphere',
 'attempt',
 'attempts',
 'attention',
 'audience',
 'average',
 'avoid',
 'away',
 'awful',
 'baby',
 'back',
 'background',
 'bad',
 'badly',
 'based',
 'basically',
 'beautiful',
 'beauty',
 'became',
 'become',
 'becomes',
 'begin',
 'beginning',
 'begins',
 'behind',
 'believable',
 'believe',
 'best',
 'better',
 'beyond',
 'big',
 'bit',
 'black',
 'blood',
 'body',
 'book',
 'bored',
 'boring',
 'box',
 'boy',
 'br',
 'br 10',
 'br br',
 'br film',
 'br movie',
 'br one',
 'br story',
 'b

In [57]:
tfidf_df = pd.DataFrame(tfidf.toarray())

In [58]:
tfidf_df.columns = nms

In [59]:
# tfidf_df.iloc[0]

In [60]:
tfidf_df['sentiment'] = 0

In [61]:
tfidf_df['sentiment'].shape

(25000,)

In [62]:
tfidf_df.loc[0:12499,'sentiment'] = 1

In [63]:
tfidf_df['sentiment'].sum()

12500

In [64]:
train_data_frame = shuffle(tfidf_df)
X = train_data_frame.iloc[:,:-1]
y = train_data_frame.iloc[:, -1]
y = to_categorical(y)

In [79]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.5, test_size=0.5, random_state=0)



model = Sequential()
model.add(Dense(128, input_shape=(908,)))
model.add(Activation('relu'))

model.add(Dense(32,kernel_regularizer=regularizers.l2(0.05)))
model.add(Activation('relu'))

model.add(Dense(8,kernel_regularizer=regularizers.l2(0.05)))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.fit(train_X, train_y, epochs=50, batch_size=256, verbose=1)

loss, accuracy = model.evaluate(test_X, test_y, verbose=0)

Epoch 1/50
12500/12500 [==============================] - 1s 42us/step - loss: 2.9593 - acc: 0.6145
Epoch 2/50
12500/12500 [==============================] - 0s 10us/step - loss: 1.7161 - acc: 0.7845
Epoch 3/50
12500/12500 [==============================] - 0s 10us/step - loss: 1.0984 - acc: 0.8377
Epoch 4/50
12500/12500 [==============================] - 0s 10us/step - loss: 0.7830 - acc: 0.8570
Epoch 5/50
12500/12500 [==============================] - 0s 10us/step - loss: 0.6210 - acc: 0.8627
Epoch 6/50
12500/12500 [==============================] - 0s 10us/step - loss: 0.5357 - acc: 0.8661
Epoch 7/50
12500/12500 [==============================] - 0s 10us/step - loss: 0.4874 - acc: 0.8694
Epoch 8/50
12500/12500 [==============================] - 0s 10us/step - loss: 0.4567 - acc: 0.8718
Epoch 9/50
12500/12500 [==============================] - 0s 10us/step - loss: 0.4351 - acc: 0.8730
Epoch 10/50
12500/12500 [==============================] - 0s 12us/step - loss: 0.4202 - acc: 0.8721

In [80]:
accuracy

0.8518400000190735

# Text Classification Using Pre Trained Word Embedding
Next we will try to predict using pre trained GloVe embeddings and see how does the result look like

You can start by using the Tokenizer utility class which can vectorize a text corpus into a list of integers. Each integer maps to a value in a dictionary that encodes the entire corpus, with the keys in the dictionary being the vocabulary terms themselves. You can add the parameter num_words, which is responsible for setting the size of the vocabulary. The most common num_words words will be then kept.

[source](https://realpython.com/python-keras-text-classification/)

In [136]:
train_full_df = pd.DataFrame(train_full_p)
train_full_df.columns = ['review']
train_full_df['sentiment']=0
train_full_df['sentiment'].iloc[0:12450]=1

In [137]:
train_full_df.sentiment.value_counts()

0    12550
1    12450
Name: sentiment, dtype: int64

In [190]:
train_full_df_shuffle = shuffle(train_full_df)
X = train_full_df_shuffle.iloc[:,:-1]
y = train_full_df_shuffle.iloc[:, -1]
y = to_categorical(y)
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [220]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(train_X.review)
train_token = tokenizer.texts_to_sequences(list(train_X.review))
test_token = tokenizer.texts_to_sequences(test_X.review)

In [223]:
maxlen_train = max([len(i) for i in train_token])
maxlen_test = max([len(i) for i in test_token])
maxlen = max(maxlen_train,maxlen_test)

In [224]:
from keras.preprocessing.sequence import pad_sequences
train_full_ps = pad_sequences(train_token, padding='post', maxlen=maxlen)
test_full_ps = pad_sequences(test_token, padding='post', maxlen=maxlen)

Since you don’t need all words, you can focus on only the words that we have in our vocabulary. Since we have only a limited number of words in our vocabulary, we can skip most of the 40000 words in the pretrained word embeddings:

In [225]:
import numpy as np
vocab_size = len(tokenizer.word_index) + 1

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [226]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('files/glove.6B.50d.txt',tokenizer.word_index, embedding_dim)

Now you are ready to use the embedding matrix in training. Let’s go ahead and use the previous network with global max pooling and see if we can improve this model. When you use pretrained word embeddings you have the choice to either allow the embedding to be updated during training or only use the resulting embedding vectors as they are.

First, let’s have a quick look how many of the embedding vectors are nonzero:

In [228]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))/ vocab_size
print(nonzero_elements)

0.8195872571782928


Let’s have a look at the performance when using the GlobalAvgPool1D layer:

In the case of max pooling you take the maximum value of all features in the pool for each feature dimension. In the case of average pooling you take the average, but max pooling seems to be more commonly used as it highlights large values.

Global max/average pooling takes the maximum/average of all features whereas in the other case you have to define the pool size. 

In [233]:
from keras import layers

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.GlobalAvgPool1D())
model.add(layers.Dense(128, activation='softmax'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 1559, 50)          3467050   
_________________________________________________________________
global_average_pooling1d_6 ( (None, 50)                0         
_________________________________________________________________
dense_75 (Dense)             (None, 128)               6528      
_________________________________________________________________
dense_76 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_77 (Dense)             (None, 2)                 130       
Total params: 3,481,964
Trainable params: 14,914
Non-trainable params: 3,467,050
_________________________________________________________________


In [235]:
model.fit(train_full_ps, train_y,
                    epochs=500,
                    verbose=True,
                    validation_data=(test_full_ps, test_y),
                    batch_size=64)
loss, accuracy = model.evaluate(train_full_ps, train_y, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_full_ps, test_y, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 20000 samples, validate on 5000 samples
Epoch 1/500
20000/20000 [==============================] - 2s 90us/step - loss: 0.6311 - acc: 0.6495 - val_loss: 0.6234 - val_acc: 0.6569
Epoch 2/500
20000/20000 [==============================] - 2s 91us/step - loss: 0.6256 - acc: 0.6575 - val_loss: 0.6166 - val_acc: 0.6667
Epoch 3/500
20000/20000 [==============================] - 2s 91us/step - loss: 0.6196 - acc: 0.6635 - val_loss: 0.6125 - val_acc: 0.6689
Epoch 4/500
20000/20000 [==============================] - 2s 91us/step - loss: 0.6138 - acc: 0.6681 - val_loss: 0.6088 - val_acc: 0.6776
Epoch 5/500
20000/20000 [==============================] - 2s 90us/step - loss: 0.6077 - acc: 0.6771 - val_loss: 0.6005 - val_acc: 0.6867
Epoch 6/500
20000/20000 [==============================] - 2s 90us/step - loss: 0.6011 - acc: 0.6833 - val_loss: 0.5929 - val_acc: 0.6989
Epoch 7/500
20000/20000 [==============================] - 2s 91us/step - loss: 0.5948 - acc: 0.6916 - val_loss: 0.5864 - v

20000/20000 [==============================] - 2s 96us/step - loss: 0.5249 - acc: 0.7420 - val_loss: 0.5191 - val_acc: 0.7449
Epoch 60/500
20000/20000 [==============================] - 2s 98us/step - loss: 0.5252 - acc: 0.7416 - val_loss: 0.5181 - val_acc: 0.7437
Epoch 61/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5243 - acc: 0.7419 - val_loss: 0.5202 - val_acc: 0.7449
Epoch 62/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5250 - acc: 0.7420 - val_loss: 0.5202 - val_acc: 0.7416
Epoch 63/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5250 - acc: 0.7399 - val_loss: 0.5210 - val_acc: 0.7398
Epoch 64/500
20000/20000 [==============================] - 2s 98us/step - loss: 0.5238 - acc: 0.7423 - val_loss: 0.5173 - val_acc: 0.7444
Epoch 65/500
20000/20000 [==============================] - 2s 96us/step - loss: 0.5239 - acc: 0.7422 - val_loss: 0.5176 - val_acc: 0.7409
Epoch 66/500
20000/20000 [==============

Epoch 118/500
20000/20000 [==============================] - 2s 96us/step - loss: 0.5149 - acc: 0.7466 - val_loss: 0.5104 - val_acc: 0.7479
Epoch 119/500
20000/20000 [==============================] - 2s 94us/step - loss: 0.5148 - acc: 0.7471 - val_loss: 0.5122 - val_acc: 0.7450
Epoch 120/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5147 - acc: 0.7486 - val_loss: 0.5118 - val_acc: 0.7480
Epoch 121/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5146 - acc: 0.7481 - val_loss: 0.5111 - val_acc: 0.7469
Epoch 122/500
20000/20000 [==============================] - 2s 96us/step - loss: 0.5144 - acc: 0.7473 - val_loss: 0.5119 - val_acc: 0.7504
Epoch 123/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5135 - acc: 0.7488 - val_loss: 0.5105 - val_acc: 0.7492
Epoch 124/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5152 - acc: 0.7478 - val_loss: 0.5124 - val_acc: 0.7506
Epoch 125/500
20000/

20000/20000 [==============================] - 2s 95us/step - loss: 0.5092 - acc: 0.7512 - val_loss: 0.5080 - val_acc: 0.7492
Epoch 177/500
20000/20000 [==============================] - 2s 94us/step - loss: 0.5092 - acc: 0.7499 - val_loss: 0.5102 - val_acc: 0.7484
Epoch 178/500
20000/20000 [==============================] - 2s 94us/step - loss: 0.5102 - acc: 0.7494 - val_loss: 0.5067 - val_acc: 0.7508
Epoch 179/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5090 - acc: 0.7508 - val_loss: 0.5102 - val_acc: 0.7486
Epoch 180/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5090 - acc: 0.7498 - val_loss: 0.5090 - val_acc: 0.7512
Epoch 181/500
20000/20000 [==============================] - 2s 94us/step - loss: 0.5095 - acc: 0.7484 - val_loss: 0.5064 - val_acc: 0.7511
Epoch 182/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5089 - acc: 0.7502 - val_loss: 0.5067 - val_acc: 0.7502
Epoch 183/500
20000/20000 [=======

20000/20000 [==============================] - 2s 95us/step - loss: 0.5063 - acc: 0.7517 - val_loss: 0.5053 - val_acc: 0.7524
Epoch 235/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5062 - acc: 0.7529 - val_loss: 0.5042 - val_acc: 0.7563
Epoch 236/500
20000/20000 [==============================] - 2s 95us/step - loss: 0.5054 - acc: 0.7529 - val_loss: 0.5061 - val_acc: 0.7565
Epoch 237/500
20000/20000 [==============================] - 2s 96us/step - loss: 0.5054 - acc: 0.7533 - val_loss: 0.5066 - val_acc: 0.7515
Epoch 238/500
20000/20000 [==============================] - 2s 100us/step - loss: 0.5061 - acc: 0.7515 - val_loss: 0.5042 - val_acc: 0.7517
Epoch 239/500
20000/20000 [==============================] - 2s 101us/step - loss: 0.5066 - acc: 0.7521 - val_loss: 0.5088 - val_acc: 0.7529
Epoch 240/500
20000/20000 [==============================] - 2s 101us/step - loss: 0.5070 - acc: 0.7508 - val_loss: 0.5063 - val_acc: 0.7535
Epoch 241/500
20000/20000 [====

20000/20000 [==============================] - 2s 101us/step - loss: 0.5038 - acc: 0.7545 - val_loss: 0.5016 - val_acc: 0.7566
Epoch 293/500
20000/20000 [==============================] - 2s 101us/step - loss: 0.5036 - acc: 0.7528 - val_loss: 0.5039 - val_acc: 0.7530
Epoch 294/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5040 - acc: 0.7547 - val_loss: 0.5075 - val_acc: 0.7507
Epoch 295/500
20000/20000 [==============================] - 2s 84us/step - loss: 0.5043 - acc: 0.7528 - val_loss: 0.5045 - val_acc: 0.7517
Epoch 296/500
20000/20000 [==============================] - 2s 89us/step - loss: 0.5034 - acc: 0.7547 - val_loss: 0.5099 - val_acc: 0.7488
Epoch 297/500
20000/20000 [==============================] - 2s 88us/step - loss: 0.5030 - acc: 0.7560 - val_loss: 0.5017 - val_acc: 0.7546
Epoch 298/500
20000/20000 [==============================] - 2s 88us/step - loss: 0.5037 - acc: 0.7530 - val_loss: 0.5041 - val_acc: 0.7540
Epoch 299/500
20000/20000 [=====

20000/20000 [==============================] - 2s 97us/step - loss: 0.5022 - acc: 0.7538 - val_loss: 0.5005 - val_acc: 0.7558
Epoch 351/500
20000/20000 [==============================] - 2s 98us/step - loss: 0.5031 - acc: 0.7531 - val_loss: 0.5056 - val_acc: 0.7552
Epoch 352/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5018 - acc: 0.7547 - val_loss: 0.5010 - val_acc: 0.7548
Epoch 353/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5019 - acc: 0.7551 - val_loss: 0.5031 - val_acc: 0.7584
Epoch 354/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5030 - acc: 0.7533 - val_loss: 0.5013 - val_acc: 0.7545
Epoch 355/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5023 - acc: 0.7545 - val_loss: 0.5042 - val_acc: 0.7577
Epoch 356/500
20000/20000 [==============================] - 2s 97us/step - loss: 0.5023 - acc: 0.7559 - val_loss: 0.5007 - val_acc: 0.7559
Epoch 357/500
20000/20000 [=======

20000/20000 [==============================] - 2s 93us/step - loss: 0.5005 - acc: 0.7568 - val_loss: 0.5015 - val_acc: 0.7560
Epoch 409/500
20000/20000 [==============================] - 2s 93us/step - loss: 0.5013 - acc: 0.7560 - val_loss: 0.5061 - val_acc: 0.7501
Epoch 410/500
20000/20000 [==============================] - 2s 93us/step - loss: 0.5004 - acc: 0.7544 - val_loss: 0.5027 - val_acc: 0.7538
Epoch 411/500
20000/20000 [==============================] - 2s 93us/step - loss: 0.5008 - acc: 0.7548 - val_loss: 0.5051 - val_acc: 0.7554
Epoch 412/500
20000/20000 [==============================] - 2s 93us/step - loss: 0.5011 - acc: 0.7550 - val_loss: 0.5055 - val_acc: 0.7522
Epoch 413/500
20000/20000 [==============================] - 2s 93us/step - loss: 0.5016 - acc: 0.7554 - val_loss: 0.5001 - val_acc: 0.7552
Epoch 414/500
20000/20000 [==============================] - 2s 93us/step - loss: 0.5006 - acc: 0.7549 - val_loss: 0.5005 - val_acc: 0.7586
Epoch 415/500
20000/20000 [=======

20000/20000 [==============================] - 2s 91us/step - loss: 0.5004 - acc: 0.7571 - val_loss: 0.5090 - val_acc: 0.7554
Epoch 467/500
20000/20000 [==============================] - 2s 89us/step - loss: 0.4995 - acc: 0.7565 - val_loss: 0.5007 - val_acc: 0.7585
Epoch 468/500
20000/20000 [==============================] - 2s 88us/step - loss: 0.4993 - acc: 0.7569 - val_loss: 0.5035 - val_acc: 0.7578
Epoch 469/500
20000/20000 [==============================] - 2s 88us/step - loss: 0.4997 - acc: 0.7557 - val_loss: 0.4987 - val_acc: 0.7588
Epoch 470/500
20000/20000 [==============================] - 2s 88us/step - loss: 0.5002 - acc: 0.7580 - val_loss: 0.4992 - val_acc: 0.7591
Epoch 471/500
20000/20000 [==============================] - 2s 88us/step - loss: 0.4991 - acc: 0.7556 - val_loss: 0.5001 - val_acc: 0.7574
Epoch 472/500
20000/20000 [==============================] - 2s 90us/step - loss: 0.4996 - acc: 0.7557 - val_loss: 0.5006 - val_acc: 0.7531
Epoch 473/500
20000/20000 [=======